In [2]:
# Cell 1: Cài đặt (chạy 1 lần)
!pip install -q sentence-transformers wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00:00:0100:01


In [3]:
# Cell 2: Imports, seed, device & W&B init
import os, random, math
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
import wandb

# --- Seed để tái lập ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# Thiết lập tokenizer parallelism để tránh warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device={device}")

wandb.finish()  
# W&B init
Name = "NgocMinh"
Model_name = "NeuFM + LLM"
Version = "1.0.0"

wandb.login(key="62e3cf4c2815c959ed2609de1d55fa0504818c4a")

# 1.1. Khởi tạo W&B run
wandb.init(
    entity="IT3190E-20242-MachinLearning",
    project="cf-electronics",
    name="HybridNeuMF_NewDS",
    config={
        "mf_dim": 32,
        "mlp_layers": [64, 32, 16, 8],
        "llm_model": "all-MiniLM-L6-v2",
        "llm_dim": 384,
        "batch_size": 1024,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "epochs": 15,
        "K": 20
    }
)
cfg = wandb.config


Using device=cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kieusontung8 (kieusontung8-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
train_df = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/train_ratings (5).csv")   # user_id,item_id,rating,...
test_df  = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/test_ratings (4).csv")
meta_df  = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/filtered_metadata (2).csv")
print(len(train_df["item_id"].unique()), len(test_df["item_id"].unique()), len(meta_df["item_id"].unique()))


2357 1126 2401


In [5]:
# Cell 3: Load & encode IDs
train_df = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/train_ratings (5).csv")   # user_id,item_id,rating,...
test_df  = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/test_ratings (4).csv")
meta_df  = pd.read_csv("/kaggle/input/ulti-chiu-cui-ri/filtered_metadata (2).csv")

# Fit trên toàn bộ user_id từ train + test
user_enc = LabelEncoder()
user_enc.fit(pd.concat([train_df["user_id"], test_df["user_id"]], ignore_index=True))
train_df["uid"] =  user_enc.transform(train_df["user_id"])
test_df["uid"]  = user_enc.transform(test_df["user_id"])

# Fit item_id từ train + test + metadata
item_enc = LabelEncoder()
item_enc.fit(pd.concat([train_df["item_id"], test_df["item_id"], meta_df["item_id"]], ignore_index=True))
train_df["iid"] = item_enc.transform(train_df["item_id"])
test_df["iid"]  = item_enc.transform(test_df["item_id"])
meta_df["iid"]  = item_enc.transform(meta_df["item_id"])

# Lấy đúng số lượng class (không dùng .nunique())
num_users = len(user_enc.classes_)
num_items = len(item_enc.classes_)

print(f"✅ Encoded: {num_users} users, {num_items} items")



✅ Encoded: 1289 users, 2401 items


In [6]:
# Cell 4: Prepare item text for LLM (concat description, features, cat egories)
meta_df["text_input"] = (
    meta_df["description"].fillna("") + " " +
    meta_df["features"].fillna("")    + " " +
    meta_df["categories"].fillna("")
)

# Reindex metadata so that row i corresponds to iid = i
meta_df = meta_df.set_index("iid").reindex(range(num_items)).fillna("")


In [7]:


# Khởi tạo mô hình LLM (không truyền device để tránh lỗi CUDA)
llm = SentenceTransformer(cfg.llm_model)

# Chuẩn bị input văn bản cho từng item
item_texts = meta_df["text_input"].tolist()  # length == num_items

# Encode bằng LLM (trả về numpy), dùng GPU mặc định nếu có
item_emb_np = llm.encode(
    item_texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True  # trả kết quả là numpy array
)

# Đảm bảo đúng shape đầu ra
assert item_emb_np.shape == (num_items, cfg.llm_dim)

# Chuyển sang torch tensor & lên GPU (nếu dùng)
item_llm_emb = torch.tensor(item_emb_np, dtype=torch.float32).to(device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [8]:
print(item_llm_emb)

tensor([[-0.0760,  0.0591, -0.0340,  ...,  0.0379,  0.0162,  0.0475],
        [-0.1133,  0.0201,  0.0159,  ..., -0.0125,  0.0535,  0.0797],
        [-0.1383,  0.0015, -0.1034,  ..., -0.0211,  0.0814,  0.0266],
        ...,
        [-0.0370,  0.0337, -0.0654,  ...,  0.0287, -0.0300,  0.0163],
        [ 0.0277, -0.0755,  0.0045,  ...,  0.0337, -0.0867,  0.0918],
        [-0.0824,  0.0717, -0.0199,  ..., -0.0560, -0.0052,  0.0227]],
       device='cuda:0')


In [9]:
# === Cell 4.5: Tạo embedding cho toàn bộ user từ review_text ===

# B1: Gộp review text theo user_id (chỉ user có review)
user_texts_partial = (
    train_df.groupby("user_id")["text"]
    .apply(lambda x: " ".join(x.dropna().astype(str)))
    .reset_index()
)

# B2: Gắn uid theo LabelEncoder
user_texts_partial["uid"] = user_enc.transform(user_texts_partial["user_id"])

# B3: Tạo mảng văn bản với số lượng = num_users, default = ""
user_text_array = [""] * num_users
user_text_dict = dict(zip(user_texts_partial["uid"], user_texts_partial["text"]))

for uid in range(num_users):
    user_text_array[uid] = user_text_dict.get(uid, "")  # fallback nếu không có review

# B4: Encode bằng SentenceTransformer
user_llm_np = llm.encode(user_text_array, batch_size=64, show_progress_bar=True)
user_llm_emb = torch.tensor(user_llm_np, dtype=torch.float32).to(device) # user_id's text được encode và chuyển sang tensor

# B5: Kiểm tra shape
assert user_llm_emb.shape == (num_users, cfg.llm_dim)
print("✅ user_llm_emb shape:", user_llm_emb.shape)


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

✅ user_llm_emb shape: torch.Size([1289, 384])


In [10]:
# === Cell 4.6: Lưu lại các embedding đã tính sau Cell 4.5 ===

save_dir = "/kaggle/working"  # Hoặc "./" nếu chạy local notebook

torch.save(item_llm_emb.cpu(), f"{save_dir}/item_llm_emb.pt")
torch.save(user_llm_emb.cpu(), f"{save_dir}/user_llm_emb.pt")

print("✅ Đã lưu: item_llm_emb.pt và user_llm_emb.pt")


✅ Đã lưu: item_llm_emb.pt và user_llm_emb.pt


In [11]:
# === Load lại các embedding đã lưu (để tránh tính lại mỗi lần) ===

item_llm_emb = torch.load(f"{save_dir}/item_llm_emb.pt").to(device)
user_llm_emb = torch.load(f"{save_dir}/user_llm_emb.pt").to(device)

print("✅ Đã load embedding vào RAM & đưa lên GPU")


✅ Đã load embedding vào RAM & đưa lên GPU


In [12]:
# === Cell 6: Dataset explicit rating (thêm user embedding) ===
class RatingDataset(Dataset):
    def __init__(self, df, item_emb, user_emb):
        self.uids = df["uid"].values
        self.iids = df["iid"].values
        self.ratings = df["rating"].values.astype(np.float32)
        self.item_emb = item_emb.cpu()
        self.user_emb = user_emb.cpu()

    def __len__(self): return len(self.uids)

    def __getitem__(self, idx):
        u = torch.LongTensor([self.uids[idx]])
        i = torch.LongTensor([self.iids[idx]])
        r = torch.FloatTensor([self.ratings[idx]])
        ll_item = self.item_emb[self.iids[idx]].unsqueeze(0)
        ll_user = self.user_emb[self.uids[idx]].unsqueeze(0)
        return u, i, ll_user, ll_item, r

# Khởi tạo loader như thường lệ
train_ds = RatingDataset(train_df, item_llm_emb, user_llm_emb)
test_ds  = RatingDataset(test_df,  item_llm_emb, user_llm_emb)

train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=cfg.batch_size, shuffle=False, num_workers=0)


In [13]:
# === Tính sampling_probs theo popularity-biased negative sampling ===
item_counts = train_df['iid'].value_counts().sort_index()

counts_tensor = torch.zeros(num_items)
counts_tensor[item_counts.index] = torch.tensor(item_counts.values, dtype=torch.float32)

sampling_probs = counts_tensor.pow(0.75)
sampling_probs /= sampling_probs.sum()  # chuẩn hóa về tổng 1
sampling_probs = sampling_probs.to(device)

# === Dataset Class ===
class ImplicitDataset(Dataset):
    def __init__(self, df_pos, num_items, item_emb, user_emb, sampling_probs, num_neg=4):
        self.pos = df_pos[["uid", "iid"]].values
        self.user_pos_dict = df_pos.groupby("uid")["iid"].apply(set).to_dict()
        self.num_items = num_items
        self.num_neg = num_neg
        self.item_emb = item_emb.to(device)
        self.user_emb = user_emb.to(device)
        self.sampling_probs = sampling_probs

    def __len__(self):
        return len(self.pos)

    def __getitem__(self, idx):
        u, i_pos = self.pos[idx]
        triplets = [(u, i_pos, 1.0)]
        neg = 0
        while neg < self.num_neg:
            i_neg = torch.multinomial(self.sampling_probs, 1).item()
            if i_neg not in self.user_pos_dict.get(u, set()):
                triplets.append((u, i_neg, 0.0))
                neg += 1

        out = []
        for uu, ii, label in triplets:
            out.append((
                torch.tensor(uu, dtype=torch.long),
                torch.tensor(ii, dtype=torch.long),
                self.user_emb[uu],
                self.item_emb[ii],
                torch.tensor(label, dtype=torch.float32)
            ))
        return out

# === Collate function ===
def collate_triplets(batch):
    u, i, lu, li, l = zip(*[t for sub in batch for t in sub])
    return (
        torch.stack(u),     # [batch_size]
        torch.stack(i),     # [batch_size]
        torch.stack(lu),    # [batch_size, emb_dim]
        torch.stack(li),    # [batch_size, emb_dim]
        torch.stack(l)      # [batch_size]
    )

# === Tạo DataLoader ===
train_ds = ImplicitDataset(
    train_df, num_items,
    item_llm_emb, user_llm_emb,
    sampling_probs=sampling_probs,
    num_neg=4
)

train_loader = DataLoader(
    train_ds,
    batch_size=cfg.batch_size,
    shuffle=True,
    collate_fn=collate_triplets,
    num_workers=0
)


In [14]:
# # === Cell 7: HybridNeuMF mở rộng với ll_user + ll_item ===
# class HybridNeuMF(nn.Module):
#     def __init__(self, n_users, n_items, mf_dim, mlp_layers, llm_dim):
#         super().__init__()
#         self.user_mf = nn.Embedding(n_users, mf_dim)
#         self.item_mf = nn.Embedding(n_items, mf_dim)
#         self.user_mlp = nn.Embedding(n_users, mlp_layers[0] // 2)
#         self.item_mlp = nn.Embedding(n_items, mlp_layers[0] // 2)

#         blocks = []
#         for d_in, d_out in zip(mlp_layers[:-1], mlp_layers[1:]):
#             blocks += [nn.Dropout(0.2), nn.Linear(d_in, d_out), nn.ReLU()]
#         self.mlp = nn.Sequential(*blocks)

#         fusion_dim = mf_dim + mlp_layers[-1] + 2 * llm_dim
#         self.predict = nn.Sequential(
#             nn.Linear(fusion_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, 1)
#         )

#     def forward(self, u, i, ll_user, ll_item):
#         mu = self.user_mf(u).squeeze(1)
#         mi = self.item_mf(i).squeeze(1)
#         cf_vec = mu * mi

#         xu = self.user_mlp(u).squeeze(1)
#         xi = self.item_mlp(i).squeeze(1)
#         mlp_vec = self.mlp(torch.cat([xu, xi], dim=1))

#         x = torch.cat([cf_vec, mlp_vec, ll_user.squeeze(1), ll_item.squeeze(1)], dim=1)
#         return self.predict(x)
import torch
import torch.nn as nn

class HybridNeuMF(nn.Module):
    def __init__(self, n_users, n_items, mf_dim, mlp_layers, llm_dim):
        super().__init__()

        # ID embeddings
        self.user_mf = nn.Embedding(n_users, mf_dim)
        self.item_mf = nn.Embedding(n_items, mf_dim)
        self.user_mlp = nn.Embedding(n_users, mlp_layers[0] // 2)
        self.item_mlp = nn.Embedding(n_items, mlp_layers[0] // 2)

        # MLP pathway
        blocks = []
        for d_in, d_out in zip(mlp_layers[:-1], mlp_layers[1:]):
            blocks += [nn.Dropout(0.2), nn.Linear(d_in, d_out), nn.ReLU()]
        self.mlp = nn.Sequential(*blocks)

        # Fusion dimensions
        self.zid_dim = mf_dim + mlp_layers[-1]
        self.zsem_dim = 2 * llm_dim
        self.fusion_input_dim = self.zid_dim + self.zsem_dim

        # Projection & gating
        self.zsem_project = nn.Linear(self.zsem_dim, self.zid_dim)
        self.gate_layer = nn.Linear(self.fusion_input_dim, 1)

        # Final prediction
        self.final_layer = nn.Sequential(
            nn.Linear(self.zid_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, u, i, ll_user, ll_item):
        # MF vector
        mu = self.user_mf(u).squeeze(1)
        mi = self.item_mf(i).squeeze(1)
        cf_vec = mu * mi

        # MLP vector
        xu = self.user_mlp(u).squeeze(1)
        xi = self.item_mlp(i).squeeze(1)
        mlp_input = torch.cat([xu, xi], dim=1)
        mlp_vec = self.mlp(mlp_input)

        # ID-based vector
        z_id = torch.cat([cf_vec, mlp_vec], dim=1)  # [B, zid_dim]

        # LLM-based semantic vector
        z_sem = torch.cat([ll_user, ll_item], dim=1)  # [B, zsem_dim]
        z_sem_proj = self.zsem_project(z_sem)         # [B, zid_dim]

        # Gated fusion
        gate_input = torch.cat([z_id, z_sem], dim=1)  # [B, fusion_input_dim]
        alpha = torch.sigmoid(self.gate_layer(gate_input))  # [B, 1]
        z_amz = alpha * z_id + (1 - alpha) * z_sem_proj      # ✅ DÙNG `z_sem_proj`

        # Final prediction
        out = self.final_layer(z_amz).squeeze(1)
        return out




In [15]:
print(cfg.llm_dim)

384


In [16]:
# === Cell 7.5: Khởi tạo model, optimizer (có regularization), loss ===

model = HybridNeuMF(
    n_users=num_users,
    n_items=num_items,
    mf_dim=cfg.mf_dim,
    mlp_layers=cfg.mlp_layers,
    llm_dim=cfg.llm_dim
).to(device)

# ✅ Thêm regularization: weight_decay > 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=cfg.lr,
    weight_decay=cfg.weight_decay  # ví dụ: 1e-4 để L2 regularization
)

criterion = criterion = nn.BCEWithLogitsLoss()



In [18]:
# # === Cell 8: Metric (precision@K, recall@K, ndcg@K...) ===
# def precision_at_k(ranked, truth, k): return len(set(ranked[:k]) & set(truth)) / k

# def recall_at_k(ranked, truth, k): return len(set(ranked[:k]) & set(truth)) / len(truth) if truth else 0.0

# def ndcg_at_k(ranked, truth, k):
#     dcg = sum(1 / math.log2(idx + 2) for idx, it in enumerate(ranked[:k]) if it in truth)
#     idcg = sum(1 / math.log2(i + 2) for i in range(min(len(truth), k)))
#     return dcg / idcg if idcg > 0 else 0.0

# def map_at_k(ranked, truth, k):
#     hits, s = 0, 0.0
#     for idx, it in enumerate(ranked[:k]):
#         if it in truth:
#             hits += 1
#             s += hits / (idx + 1)
#     return s / len(truth) if truth else 0.0

# def mrr_at_k(ranked, truth, k):
#     for idx, it in enumerate(ranked[:k]):
#         if it in truth: return 1 / (idx + 1)
#     return 0.0

# @torch.no_grad()
# def evaluate_full(model, train_df, test_df, K, loader):
#     model.eval()
#     train_map = train_df.groupby("uid")["iid"].apply(set).to_dict()
#     test_map = test_df.groupby("uid")["iid"].apply(list).to_dict()
#     P, R, N, AP, MRR = [], [], [], [], []
#     for u, truth in test_map.items():
#         if not truth: continue
#         users = torch.LongTensor([u] * num_items).to(device)
#         items = torch.arange(num_items).to(device)
#         ll_item = item_llm_emb.to(device)
#         ll_user = user_llm_emb[u].unsqueeze(0).repeat(num_items, 1).to(device)
#         scores = model(users, items, ll_user.unsqueeze(1), ll_item.unsqueeze(1)).view(-1).cpu().numpy()
#         for it in train_map.get(u, []):
#             scores[it] = -np.inf
#         ranked = np.argsort(-scores)
#         P.append(precision_at_k(ranked, truth, K))
#         R.append(recall_at_k(ranked, truth, K))
#         N.append(ndcg_at_k(ranked, truth, K))
#         AP.append(map_at_k(ranked, truth, K))
#         MRR.append(mrr_at_k(ranked, truth, K))
#     return {
#         "Precision@K": np.mean(P),
#         "Recall@K": np.mean(R),
#         "NDCG@K": np.mean(N),
#         "MAP@K": np.mean(AP),
#         "MRR@K": np.mean(MRR),
#     }
# === Cell 8: Metric functions (precision@K, recall@K, ndcg@K, MAP@K, MRR@K, HR@K) and evaluate_full ===
import math
import numpy as np
import torch

def precision_at_k(ranked_list, ground_truth_set, k):
    """
    Precision@K = |{items in ranked_list[:k]} ∩ ground_truth_set| / k
    """
    if k == 0:
        return 0.0
    num_hit = len(set(ranked_list[:k]) & ground_truth_set)
    return num_hit / k

def recall_at_k(ranked_list, ground_truth_set, k):
    """
    Recall@K = |{items in ranked_list[:k]} ∩ ground_truth_set| / |ground_truth_set|
    """
    if not ground_truth_set:
        return 0.0
    num_hit = len(set(ranked_list[:k]) & ground_truth_set)
    return num_hit / len(ground_truth_set)

def ndcg_at_k(ranked_list, ground_truth_set, k):
    """
    NDCG@K:
      DCG = sum_{i=0..k-1} (1 / log2(i+2)) if ranked_list[i] in ground_truth_set
      IDCG = sum_{i=0..min(|ground_truth_set|,k)-1} (1 / log2(i+2))
      NDCG = DCG / IDCG
    """
    dcg = 0.0
    for idx, item in enumerate(ranked_list[:k]):
        if item in ground_truth_set:
            dcg += 1.0 / math.log2(idx + 2)
    ideal_count = min(len(ground_truth_set), k)
    if ideal_count == 0:
        return 0.0
    idcg = sum(1.0 / math.log2(i + 2) for i in range(ideal_count))
    return dcg / idcg

def map_at_k(ranked_list, ground_truth_set, k):
    """
    MAP@K:
      For each position i in top-K, if ranked_list[i] in ground_truth_set:
        hits += 1
        sum_precisions += hits / (i+1)
      Then divide by |ground_truth_set|.
    """
    if not ground_truth_set:
        return 0.0
    hits = 0
    sum_precisions = 0.0
    for idx, item in enumerate(ranked_list[:k]):
        if item in ground_truth_set:
            hits += 1
            sum_precisions += hits / (idx + 1)
    return sum_precisions / len(ground_truth_set)

def mrr_at_k(ranked_list, ground_truth_set, k):
    """
    MRR@K:
      Return 1 / (rank) of the first relevant item in top-K; else 0.
    """
    for idx, item in enumerate(ranked_list[:k]):
        if item in ground_truth_set:
            return 1.0 / (idx + 1)
    return 0.0

def hit_rate_at_k(ranked_list, ground_truth_set, k):
    """
    HR@K (Hit Rate): 1 if at least one item in top-K is relevant, else 0.
    """
    return int(bool(set(ranked_list[:k]) & ground_truth_set))

@torch.no_grad()


def evaluate_full(model, train_df, test_df, K, loader, device, num_items, user_llm_emb, item_llm_emb):
    """
    model: recommendation model returning a score for (user, item)
    train_df: DataFrame with columns ["uid","iid"] (user-item interactions in train)
    test_df: DataFrame with columns ["uid","iid"] (user-item interactions in test; here we treat 
             every interaction as positive for ranking)
    K: the cutoff for @K
    loader: not used here (can pass None)
    device: torch device
    num_items: total number of items
    user_llm_emb: tensor of shape (num_users, embed_dim)
    item_llm_emb: tensor of shape (num_items, embed_dim)
    """
    model.eval()
    # Build a map: user -> set of items in train (for filtering)
    train_map = train_df.groupby("uid")["iid"].apply(set).to_dict()

    # Build ground_truth_map: user -> set of items in test (all are positives)
    ground_truth_map = test_df.groupby("uid")["iid"].apply(set).to_dict()

    precisions = []
    recalls = []
    ndcgs = []
    hrs = []

    for u, truth_set in ground_truth_map.items():
        if not truth_set:
            continue

        # Compute scores for all items for user u
        users = torch.LongTensor([u] * num_items).to(device)   # shape: (num_items,)
        items = torch.arange(num_items).to(device)              # shape: (num_items,)
        ll_user = user_llm_emb[users].to(device)                # shape: (num_items, embed_dim)
        ll_item = item_llm_emb[items].to(device)                # shape: (num_items, embed_dim)
        scores = model(users, items, ll_user, ll_item).view(-1).cpu().numpy()

        # Filter out items that appear in train for this user
        for it in train_map.get(u, set()):
            scores[it] = -np.inf

        ranked = np.argsort(-scores)  # Indices sorted descending by score

        # Compute metrics @ K
        precisions.append(precision_at_k(ranked, truth_set, K))
        recalls.append(recall_at_k(ranked, truth_set, K))
        ndcgs.append(ndcg_at_k(ranked, truth_set, K))
        hrs.append(hit_rate_at_k(ranked, truth_set, K))

    return {
        f"Precision@{K}": np.mean(precisions) if precisions else 0.0,
        f"Recall@{K}":    np.mean(recalls)    if recalls    else 0.0,
        f"NDCG@{K}":      np.mean(ndcgs)      if ndcgs      else 0.0,
        f"HR@{K}":        np.mean(hrs)        if hrs        else 0.0,
    }


In [ ]:
pip install -q torch-optimizer


In [ ]:
# === Training loop ===
best_ndcg = 0.0
patience = 15
counter = 0
results = []

for ep in range(1, cfg.epochs + 1):
    model.train()
    total_loss = 0.0

    for u, i, ll_user, ll_item, r in tqdm(train_loader, desc=f"Epoch {ep}"):
        u = u.to(device)
        i = i.to(device)
        ll_user = ll_user.to(device)
        ll_item = ll_item.to(device)
        r = r.to(device).float().view(-1)

        optimizer.zero_grad()
        pred = model(u, i, ll_user, ll_item)
        if pred.dim() > 1:
            pred = pred.view(-1)
        loss = criterion(pred, r)
        loss.backward()

        # ❌ KHÔNG dùng gradient clipping
        optimizer.step()
        total_loss += loss.item() * r.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # === Evaluation (metrics@K) ===
    metrics = evaluate_full(
        model,
        train_df,
        test_df,
        cfg.K,
        None,
        device,
        num_items,
        user_llm_emb,
        item_llm_emb
    )

    metrics["Train Loss"] = train_loss
    metrics["epoch"] = ep
    wandb.log(metrics)
    results.append(metrics)

    print(
        f"Epoch {ep} — "
        f"Precision@{cfg.K}={metrics[f'Precision@{cfg.K}']:.4f}  "
        f"Recall@{cfg.K}={metrics[f'Recall@{cfg.K}']:.4f}  "
        f"NDCG@{cfg.K}={metrics[f'NDCG@{cfg.K}']:.4f}  "
        f"HR@{cfg.K}={metrics[f'HR@{cfg.K}']:.4f}"
    )

    # === Early stopping based on best NDCG@K ===
    ndcg = metrics[f"NDCG@{cfg.K}"]
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        counter = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        counter += 1
        if counter >= patience:
            print(f"\n🛑 Early stopping at epoch {ep} — best NDCG@{cfg.K} = {best_ndcg:.4f}")
            break

Epoch 1:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 1 — Precision@20=0.0010  Recall@20=0.0039  NDCG@20=0.0020  HR@20=0.0194


Epoch 2:   0%|          | 0/39 [00:00<?, ?it/s]